#### LLM Zoomcamp 2.8 - Ollama + Elastic in Docker Compose

In [1]:
from openai import OpenAI

In [2]:
client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [4]:
!pip install elasticsearch

550.48s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.2/480.2 kB 9.7 MB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 kB 1.5 MB/s eta 0:00:00.0 MB/s eta 0:00:01

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
from elasticsearch import Elasticsearch

In [4]:
es_client = Elasticsearch('http://localhost:9200') 

In [5]:
es_client.info()

ObjectApiResponse({'name': '6201952bb194', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'h6_D5IUvRICrhJT5XUhDnA', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [7]:
!curl -X DELETE "localhost:9200/course-questions"

{"acknowledged":true}

In [8]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [9]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [10]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [ ]:
!pip install --upgrade notebook jupyterlab

In [ ]:
!pip install --upgrade ipywidgets

In [11]:
from tqdm.auto import tqdm

In [12]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [13]:
def elastic_search(query):
    search_query = {
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    try:
        response = es_client.search(index=index_name, body=search_query
        )
        result_docs = []
        for hit in response['hits']['hits']:
            result_docs.append(hit['_source'])
    except Exception as e:
        print(f"An error occurred: {str(e)}")
    
    return result_docs 

In [14]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    response = client.chat.completions.create(
        model='phi3',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content


In [15]:
def rag(query):
    search_result = elastic_search(query)
    prompt = build_prompt(query, search_result)
    answer = llm(prompt)
    return answer

In [16]:
query = 'I just discovered the course. Can I still join it?'
rag(query)

" As a course teaching assistant, I can confirm that if you find the Data Engineering Bootcamp valuable and relevant to your interests or career goals, there are multiple ways for you to contribute:\n\nFirstly, star the GitHub repository associated with this bootcamp as well as share it on social media platforms where like-minded individuals gather. Your support will not only show appreciation but also help maintain a vibrant learning community around these courses and projects that can lead directly back to their creators for more opportunities or improvements in future iterations of the course materials.\n\nIf you spot any areas within this repository, whether it be textual clarity on explanations/descriptions or structural enhancements like organization into separate modules with clear navigation links between them, feel empowered to fork and create a Pull Request (PR). By doing so not only do you contribute back but also get involved in shaping the learning materials directly. It's

In [17]:
print(_)

 As a course teaching assistant, I can confirm that if you find the Data Engineering Bootcamp valuable and relevant to your interests or career goals, there are multiple ways for you to contribute:

Firstly, star the GitHub repository associated with this bootcamp as well as share it on social media platforms where like-minded individuals gather. Your support will not only show appreciation but also help maintain a vibrant learning community around these courses and projects that can lead directly back to their creators for more opportunities or improvements in future iterations of the course materials.

If you spot any areas within this repository, whether it be textual clarity on explanations/descriptions or structural enhancements like organization into separate modules with clear navigation links between them, feel empowered to fork and create a Pull Request (PR). By doing so not only do you contribute back but also get involved in shaping the learning materials directly. It's coll